# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 16.7 MB/s eta 0:00:00


In [3]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Load trainset

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [5]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

# Load adv samples

In [6]:
feature_list = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg',
       'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg',
       'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
       'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

In [7]:
pgd_path = '/content/drive/MyDrive/Dataset/insdn/adv/24_feature/pgd/pgd_mlp_bin_feature'
#pgd_label_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/pgd/adv_pgd_target.csv'

In [8]:
pgd_df = pd.read_csv(pgd_path)

In [9]:
pgd_df.columns = feature_list
pgd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      55151 non-null  float64
 1   Tot Fwd Pkts       55151 non-null  float64
 2   Tot Bwd Pkts       55151 non-null  float64
 3   TotLen Fwd Pkts    55151 non-null  float64
 4   TotLen Bwd Pkts    55151 non-null  float64
 5   Fwd Pkt Len Max    55151 non-null  float64
 6   Fwd Pkt Len Min    55151 non-null  float64
 7   Fwd Pkt Len Mean   55151 non-null  float64
 8   Fwd Pkt Len Std    55151 non-null  float64
 9   Bwd Pkt Len Max    55151 non-null  float64
 10  Bwd Pkt Len Min    55151 non-null  float64
 11  Bwd Pkt Len Mean   55151 non-null  float64
 12  Bwd Pkt Len Std    55151 non-null  float64
 13  Flow Byts/s        55151 non-null  float64
 14  Flow Pkts/s        55151 non-null  float64
 15  Flow IAT Mean      55151 non-null  float64
 16  Flow IAT Std       551

In [ ]:
#pgd_label = pd.read_csv(pgd_label_path)
#pgd_label.info()

# Load DL-based IDS model

In [10]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/insdn/insdn_mlp_bin.h5', compile=True)

# SHAP

In [11]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100))

In [12]:
pgd_1000 = pgd_df.head(1000)

In [13]:
shap_values = explainer.shap_values(pgd_1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
print(shap_values[0])

[[ 0.00497214 -0.00619488  0.00545326 ...  0.01922717  0.00607816
   0.00794432]
 [ 0.          0.00794476  0.         ...  0.02435872  0.
   0.        ]
 [ 0.         -0.01134098 -0.00499612 ...  0.01184063 -0.00545645
  -0.01563582]
 ...
 [-0.00639864  0.          0.01000043 ...  0.03353839 -0.04039139
   0.00592127]
 [ 0.00341657  0.          0.00258532 ...  0.02890894  0.00647786
   0.00315613]
 [ 0.          0.         -0.0093505  ...  0.01084417  0.
   0.01120318]]


In [15]:
type(shap_values[0])

numpy.ndarray

In [16]:
data_df = pd.DataFrame(shap_values[0])

In [17]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.004972,-0.006195,0.005453,0.001362,0.00000,-0.002722,0.002954,0.000000,0.005059,0.000000,...,0.000000,0.0,0.045749,-0.018369,0.017977,0.058000,0.007208,0.019227,0.006078,0.007944
1,0.000000,0.007945,0.000000,0.000000,0.00000,0.001632,0.000000,0.013999,0.000000,-0.004657,...,0.000000,0.0,0.026168,-0.077192,0.034185,0.043720,-0.011041,0.024359,0.000000,0.000000
2,0.000000,-0.011341,-0.004996,-0.004294,0.00000,0.000000,0.005749,0.004760,0.005219,-0.003560,...,0.005586,0.0,0.000000,-0.072904,0.000000,0.045193,-0.003338,0.011841,-0.005456,-0.015636
3,0.000000,0.009279,0.000000,-0.006054,-0.00639,0.000000,0.002281,0.000000,0.004672,0.000000,...,0.005573,0.0,0.028532,-0.028519,0.032431,0.053298,0.006366,0.017005,-0.013403,-0.034657
4,-0.059752,0.008565,-0.006617,0.000000,0.00000,0.000000,0.000000,-0.003214,-0.012403,0.000000,...,-0.017102,0.0,0.052823,-0.267211,0.059098,0.065598,-0.154705,0.000000,-0.163087,-0.201283


In [ ]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_pgd/1000_samples_[0].csv', index=False)

In [18]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.004972,-0.006195,0.005453,0.001362,0.00000,-0.002722,0.002954,0.000000,0.005059,0.000000,...,0.000000,0.0,0.045749,-0.018369,0.017977,0.058000,0.007208,0.019227,0.006078,0.007944
1,0.000000,0.007945,0.000000,0.000000,0.00000,0.001632,0.000000,0.013999,0.000000,-0.004657,...,0.000000,0.0,0.026168,-0.077192,0.034185,0.043720,-0.011041,0.024359,0.000000,0.000000
2,0.000000,-0.011341,-0.004996,-0.004294,0.00000,0.000000,0.005749,0.004760,0.005219,-0.003560,...,0.005586,0.0,0.000000,-0.072904,0.000000,0.045193,-0.003338,0.011841,-0.005456,-0.015636
3,0.000000,0.009279,0.000000,-0.006054,-0.00639,0.000000,0.002281,0.000000,0.004672,0.000000,...,0.005573,0.0,0.028532,-0.028519,0.032431,0.053298,0.006366,0.017005,-0.013403,-0.034657
4,-0.059752,0.008565,-0.006617,0.000000,0.00000,0.000000,0.000000,-0.003214,-0.012403,0.000000,...,-0.017102,0.0,0.052823,-0.267211,0.059098,0.065598,-0.154705,0.000000,-0.163087,-0.201283


In [19]:
data_dict = {}
for idx in range(0, 1000):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [20]:
print(data_dict)

{'Adv sample 0': ['Active Min', 'Init Bwd Win Byts', 'Bwd IAT Min', 'Active Mean', 'Pkt Len Min', 'Fwd Byts/b Avg', 'Idle Std', 'Down/Up Ratio', 'Active Max', 'Bwd IAT Std'], 'Adv sample 1': ['SYN Flag Cnt', 'Bwd IAT Min', 'Init Bwd Win Byts', 'Active Min', 'Fwd IAT Min', 'Active Max', 'Bwd IAT Std', 'Active Mean', 'Idle Std', 'Fwd Pkt Len Mean'], 'Adv sample 2': ['SYN Flag Cnt', 'Init Bwd Win Byts', 'Active Min', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Std', 'Pkt Len Min', 'Fwd Byts/b Avg', 'Bwd IAT Max', 'Bwd IAT Min'], 'Adv sample 3': ['SYN Flag Cnt', 'Active Min', 'Init Bwd Win Byts', 'Fwd IAT Min', 'Active Max', 'Active Mean', 'Bwd IAT Mean', 'Pkt Len Min', 'Bwd IAT Std', 'Bwd IAT Max'], 'Adv sample 4': ['ACK Flag Cnt', 'Bwd IAT Std', 'Active Min', 'Active Max', 'Active Mean', 'Init Bwd Win Byts', 'Flow IAT Mean', 'Pkt Len Max', 'Fwd IAT Min', 'Bwd Pkt Len Min'], 'Adv sample 5': ['Active Mean', 'Bwd IAT Max', 'Bwd IAT Mean', 'Fwd IAT Min', 'Init Bwd Win Byts', 'Fwd Blk Rate Avg', 

In [21]:
len(shap_values[0])

1000

In [22]:
len(data_dict)

1000

# Detection phase

In [23]:
import os
import pandas as pd

In [24]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/InSDN/MLP_top24_InSDN_bin.csv')
whitelist_df.head(24)

,Feature,Frequency
0,Init Bwd Win Byts,684
1,Idle Max,604
2,Bwd Pkt Len Max,571
3,Idle Mean,535
4,Idle Min,535
5,Bwd Pkt Len Mean,462
6,Bwd Pkt Len Std,444
7,Pkt Len Min,383
8,Tot Fwd Pkts,364
9,Bwd Seg Size Avg,358


## Detection rate (top 24) = 1000/1000

In [31]:
whitelist = whitelist_df.iloc[:, 0][:24].tolist()
print(whitelist)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Idle Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Pkt Len Min', 'Tot Fwd Pkts', 'Bwd Seg Size Avg', 'Bwd IAT Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Tot', 'SYN Flag Cnt', 'Flow Pkts/s', 'Bwd IAT Std', 'Bwd Pkt Len Min', 'Fwd Pkt Len Min', 'Flow Duration', 'Down/Up Ratio', 'Flow IAT Min', 'Fwd Pkt Len Std']


In [37]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:1000


## Detection rate (top 38) = 992/1000

In [38]:
whitelist2 = whitelist_df.iloc[:, 0][:38].tolist()
print(whitelist2)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Idle Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Pkt Len Min', 'Tot Fwd Pkts', 'Bwd Seg Size Avg', 'Bwd IAT Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Tot', 'SYN Flag Cnt', 'Flow Pkts/s', 'Bwd IAT Std', 'Bwd Pkt Len Min', 'Fwd Pkt Len Min', 'Flow Duration', 'Down/Up Ratio', 'Flow IAT Min', 'Fwd Pkt Len Std', 'Fwd Pkts/s', 'Active Min', 'Fwd IAT Std', 'Flow IAT Mean', 'Fwd IAT Mean', 'Bwd Header Len', 'Pkt Len Var', 'Active Mean', 'Fwd IAT Tot', 'Active Max', 'Idle Std', 'Active Std', 'Flow IAT Std', 'Subflow Bwd Pkts']


In [39]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:992
